<a href="https://colab.research.google.com/github/changyuhsin1999/RAG_with_LLM/blob/main/Note.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Remove Colab default sample_data
!rm -r ./sample_data

# Clone GitHub files to colab workspace
repo_name = "RAG_with_LLM" # repo name
git_path = 'https://github.com/changyuhsin1999/RAG_with_LLM.git'
!git clone "{git_path}"

Cloning into 'RAG_with_LLM'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), done.


In [2]:
!pip install beautifulsoup4
!pip install openai
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 2.9 MB/s eta 0:00:00


In [3]:
import openai
import csv
#import PyPDF2
import numpy as np
#from scipy.spatial import distance_matrix
from bs4 import BeautifulSoup
import warnings
import os
from urllib.request import urlopen
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import os
import sqlite3

In [4]:
#Book url
url = "https://www.gutenberg.org/files/22381/22381-h/22381-h.htm"

In [5]:
def split_text(text, chunk_size):
    text_chunks = []
    text_length = len(text)
    start = 0
    while start < text_length:
        end = start + chunk_size
        # Ensure we do not go beyond the text's length
        if end > text_length:
            end = text_length
        else:
            # Find the last period in the current chunk
            last_period_index = text.rfind('.', start, end)
            if last_period_index != -1:
                end = last_period_index + 1  # Include the period in the chunk
            else:
                # If there's no period, we extend to the next period outside the current chunk
                next_period_index = text.find('.', end)
                if next_period_index != -1:
                    end = next_period_index + 1  # Include the period in the chunk
                else:
                    # If there are no more periods, we take the rest of the text
                    end = text_length
        chunk = text[start:end]
        text_chunks.append(chunk)
        start = end
    return text_chunks

In [6]:
def create_embeddings(text_chunks, model="text-embedding-ada-002"):
    embeddings = []
    try:
        prepared_chunks = [chunk.replace("\n", " ") for chunk in text_chunks]
        response = openai.Embedding.create(input=prepared_chunks, model=model)
        if response and "data" in response:
            for data in response["data"]:
                embeddings.append(data["embedding"])
        return embeddings
    except Exception as e:
        print(f"Error creating embeddings: {e}")
        return None

In [7]:
def read_html_file(file_path):
  html = urlopen(url).read()
  soup = BeautifulSoup(html, "html.parser")
  text = soup.get_text()
    # Split text into words and join with a single space to remove extra whitespace
  cleaned_text = ' '.join(text.split())
  return cleaned_text

In [8]:
def initialize_db(db_path='embeddings.db'):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    # Create a table with id, text, and embedding (stored as BLOB)
    c.execute('''CREATE TABLE IF NOT EXISTS sentence_embeddings
                 (id INTEGER PRIMARY KEY, text TEXT, embedding BLOB)''')
    conn.commit()
    return conn

In [9]:
def store_embeddings(conn, texts, embeddings):
    c = conn.cursor()
    for text, embedding in zip(texts, embeddings):
        # Convert numpy array to bytes for storage
        embedding_blob = embedding.tobytes()
        c.execute("INSERT INTO sentence_embeddings (text, embedding) VALUES (?, ?)",
                  (text, embedding_blob))
    conn.commit()

In [10]:
model = SentenceTransformer('all-MiniLM-L6-v2')
cleaned_text = read_html_file(url)
text_chunks = split_text(cleaned_text, 300)
# Generate embeddings
embeddings = model.encode(text_chunks)

# Initialize database and create table if not exists
conn = initialize_db()

# Store the embeddings and the sentences in the database
store_embeddings(conn, text_chunks, embeddings)

# Close the connection
conn.close()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
user_query = "what is the story about Orpheus and Eurydice"

# Calculate embedding for the user query
user_query_embedding = model.encode([user_query])

In [17]:
conn = sqlite3.connect('embeddings.db')
cursor = conn.cursor()

# Fetch all embeddings and their corresponding texts from the database
cursor.execute("SELECT embedding, text FROM sentence_embeddings")
rows = cursor.fetchall()

# Convert embeddings from database (assuming they are stored as lists or similar structure) and calculate cosine similarity
db_embeddings = np.array([np.frombuffer(row[0], dtype=np.float32) for row in rows])
cos_similarities = cosine_similarity(user_query_embedding, db_embeddings)

# Get the indices of the top 3 most similar embeddings
top_5_indices = np.argsort(cos_similarities[0])[-5:]

# Retrieve the corresponding texts for the top 3 embeddings
top_5_texts = [rows[i][1] for i in top_5_indices]

# Print or return the top 3 similar texts
for text in top_5_texts:
    print(text)

 Eurydice died of the wound, and her sorrowing husband filled the groves and valleys with his piteous and unceasing lamentations.
 The glance was fatal, and destroyed all his hopes of happiness; for, as he yearningly stretched out his arms to embrace her, she was caught back, and vanished from his sight for ever. The grief of Orpheus at this second loss was even more intense than before, and he now avoided all human society.
 Orpheus gladly promised to comply with this injunction, and, followed by Eurydice, ascended the steep and gloomy path which led to the realms of life and light.
 The madly rushing torrents stopped their rapid course, and the very mountains and trees moved from their places at the sound of his entrancing melodies. Orpheus became united to a lovely nymph named Eurydice, the daughter of the sea-god Nereus, whom he fondly loved.
 But it was only short-lived; for Aristæus,[31] the half-brother of Orpheus, having fallen in love with the beautiful Eurydice, forcibly ende

In [18]:
def query_llm_with_prompt(top_5_texts):

    prompt = f"Based on the following information: \n1. {top_5_texts[0]}\n2. {top_5_texts[1]}\n3. {top_5_texts[2]}\n4. {top_5_texts[3]}\n5. {top_5_texts[4]}\nGenerate an answer to this user question: {user_query}."
    messages=[{"role": "assistant", "content": "You are an expert in Greek myth, please answer questions with more information about this Greek story, do not make up your own story"}, {"role": "user", "content": prompt}]
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages
    )
    response = completion.choices[0].message.content
    return response

In [19]:
response = query_llm_with_prompt(top_5_texts)
print(response)

Orpheus was a legendary musician in Greek mythology, known for his enchanting music. The tragic story of Orpheus and Eurydice begins with their blissful marriage, which is cut short when Eurydice dies of a snake bite. Orpheus, utterly grief-stricken, plays mournful tunes that move even the natural world. His music was so captivating that he was able to charm the creatures of the world, including the gods and the underworld.

Orpheus descends to the underworld to retrieve Eurydice, and his music softens the hearts of Hades and Persephone. They agree to let Eurydice return to the world of the living under one condition: Orpheus must lead her out of the underworld without looking back at her until they reach the surface. But unable to resist the temptation, Orpheus glances back at Eurydice just before they reach the surface, and she is pulled back into the underworld, lost to him forever. 

The grief-stricken Orpheus wanders the earth, shunning human company, and ultimately meets a tragic